In [1]:
%reload_ext autoreload

In [2]:
#!pip install wandb

In [3]:
# Imports

%load_ext autoreload
%autoreload 2

%matplotlib inline
import numpy as np
import random
import csv
import pandas as pd
import matplotlib.pyplot as plt
import wandb

from fastai.imports import *
from fastai.structured import *
from matplotlib import pyplot
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV
from scipy.stats import trim_mean, kurtosis
from scipy.stats.mstats import mode, gmean, hmean
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn.externals import joblib
from sklearn.metrics import f1_score,\
    accuracy_score, confusion_matrix,\
    precision_score, recall_score,\
    roc_curve, roc_auc_score,\
    cohen_kappa_score, mean_absolute_error,\
    precision_recall_curve, auc,\
    average_precision_score

set_plot_sizes(12,14,16)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/justinhennessy/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
import csv

sys.path.append('../lib')
import utils
import preprocessing

In [29]:
# Load in data

PATH = "../data/"
df = pd.read_csv(f'{PATH}churn.csv', low_memory=False, 
                     parse_dates=['canceldate', 'licence_registration_date', 'golive_date'])

In [30]:
df[df.username == 'N061944']

,username,licence_registration_date,licence_status,url,golive_date,canceldate,plan,days_active,account_status,customer_account_status,...,interactions_total,interactions_completed,interactions_no_response,interactions_no_onboarding,interactions_completed_training,churn_concern_cases_total,churn_concern_cases_open,total_churn_concern_cases_age,total_churn_concern_cases_age_avg,churned
6003,N061944,NaT,LIVE,www.music440.com.au,NaT,NaT,PLAN-S-19Q1,NaN,Active,Good,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,403.0,403.0,0


In [31]:
df.columns

Index(['username', 'licence_registration_date', 'licence_status', 'url',
       'golive_date', 'canceldate', 'plan', 'days_active', 'account_status',
       'customer_account_status', 'nps', 'last_login_days', 'golive_days',
       'last_month_total', 'mtd_total', 'last_month_open', 'mtd_open',
       'last_month_closed', 'mtd_closed', 'last_month_age_hours',
       'mtd_age_hours', 'month_total_avg', 'open_month_avg',
       'closed_month_avg', 'age_hours_month_avg', 'interactions_total',
       'interactions_completed', 'interactions_no_response',
       'interactions_no_onboarding', 'interactions_completed_training',
       'churn_concern_cases_total', 'churn_concern_cases_open',
       'total_churn_concern_cases_age', 'total_churn_concern_cases_age_avg',
       'churned'],
      dtype='object')

In [32]:
# Delete rubbish rows
rubbish_urls = [ 'dev-spotlightmy.neto.com.au', 'picknpack-demo.neto.com.au' ]

for url in rubbish_urls:
    loc = df[df.url == url].index[0]
    df = df.drop([loc], axis=0)

drop_columns = ['url', 'golive_date']

df_raw = df.drop(columns=drop_columns)
    
# Delete rubbish usernames
rubbish_usernames = ['c8kLmHScrYaUKAz', 'TNRl7FnatO5CzMZ', 'rCKGN1JlpnRssbH']

for username in rubbish_usernames:
    loc = df_raw[df_raw.username == username].index[0]
    df_raw = df_raw.drop([loc], axis=0)

In [33]:
print(len(df_raw.loc[df_raw.churned == 0]), df_raw.loc[df_raw.churned == 0].shape)

df_processed = preprocessing.prepare_data(df_raw.loc[(df_raw.licence_status != "STAGING") & (df_raw.licence_status != "PARTNER") & (df_raw.plan != "CUSTOM") & (df_raw.plan != "TRIAL")])
print(len(df_processed), df_processed.shape)

2435 (2435, 33)
Sorting dataframe by licence_registration_date ...
Defaulting 'last_login_days' for missing values to the plans mean or platform medeian ...
  Platform 'last_login_days' median: 197.0
  Platform 'last_login_days' mean: 199.39724137931034
  List records that still have NaN in them for feature 'last_login_days' ...
  0
Defaulting 'last_month_total' for missing values to the plans mean or platform medeian ...
  Platform 'last_month_total' median: 0.0
  Platform 'last_month_total' mean: 1.5089340727048675


../lib/preprocessing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['plan'][i] = "no_plan"
../lib/preprocessing.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['plan_not_set'][i] = 1


  List records that still have NaN in them for feature 'last_month_total' ...
  0
Defaulting 'mtd_total' for missing values to the plans mean or platform medeian ...
  Platform 'mtd_total' median: 0.0
  Platform 'mtd_total' mean: 0.2350585335797905
  List records that still have NaN in them for feature 'mtd_total' ...
  0
Defaulting 'last_month_open' for missing values to the plans mean or platform medeian ...
  Platform 'last_month_open' median: 0.0
  Platform 'last_month_open' mean: 0.21626617375231053
  List records that still have NaN in them for feature 'last_month_open' ...
  0
Defaulting 'mtd_open' for missing values to the plans mean or platform medeian ...
  Platform 'mtd_open' median: 0.0
  Platform 'mtd_open' mean: 0.10166358595194085
  List records that still have NaN in them for feature 'mtd_open' ...
  0
Defaulting 'last_month_closed' for missing values to the plans mean or platform medeian ...
  Platform 'last_month_closed' median: 0.0
  Platform 'last_month_closed' mean

In [34]:
print(len(df_processed), df_processed.shape)
df_processed = df_processed.loc[df_processed.licence_status == "LIVE"]
df_processed = df_processed.drop(columns=['licence_status'])
print(len(df_processed), df_processed.shape)

6331 (6331, 113)
2401 (2401, 112)


In [35]:
print(len(df_processed), df_processed.shape)

df_data, y_data, nas = proc_df(df_processed, 'churned')

print(len(df_data))
print(df_data.shape)
print(nas)

2401 (2401, 112)
2401
(2401, 126)
{'golive_days': 42594.0, 'interactions_total': 2.0, 'interactions_completed': 2.0, 'interactions_no_response': 0.0, 'interactions_no_onboarding': 0.0, 'interactions_completed_training': 0.0, 'churn_concern_cases_total': 1.0, 'churn_concern_cases_open': 0.0, 'total_churn_concern_cases_age_avg': 334.0, 'licence_registration_Year': 2017.0, 'licence_registration_Month': 7.0, 'licence_registration_Week': 27.0, 'licence_registration_Day': 6.0, 'licence_registration_Dayofweek': 3.0, 'licence_registration_Dayofyear': 187.0}


In [36]:
# this is where we "checkout" our latest model, likely done at container build time
joblib_file = "./wandb/churn_model.pkl"

# Load from file
churn_model = joblib.load(joblib_file)

In [37]:
predictions = churn_model.predict(df_data)
probability = churn_model.predict_proba(df_data)[:,1]
len(probability), len(predictions), len(df_processed)

(2401, 2401, 2401)

In [38]:
df_predictions = pd.DataFrame(predictions)
df_probability = pd.DataFrame(probability)

In [39]:
#df_output = df_processed[df_processed.churned == 0].copy()
#df_output = df_processed[df_processed.churned == 0].copy()

In [40]:
#df_output['prediction'] = predictions
#df_output['probability'] = probability
#df_output.prediction, y_data

df_data['prediction'] = predictions
df_data['probability'] = probability

accuracy_score(y_data,df_data['prediction'])

#len(y_data), len(df_data['prediction'])
#y_data[:20], df_data
#df_data[y_data != df_data.prediction]

0.9266972094960433

In [41]:
# Calculate churn concern %
churn_concern_count = len(df_data.loc[df_data['prediction'] == 1])
total_active = len(df_data)
churn_concern_percent = round((churn_concern_count / total_active) * 100, 2)
print(f"Churn concern count: {churn_concern_count} of {total_active} ({churn_concern_percent}%)")

Churn concern count: 176 of 2401 (7.33%)


In [42]:
# Sort churn concerns by probability
df_sorted = df_data.loc[df_data['prediction'] == 1].sort_values(by=['probability'], ascending=False)

In [43]:
#df_output.annual_revenue_log.var(), df_output.annual_revenue_log.skew(), df_output.annual_revenue_log.mad(), df_output.annual_revenue_log.mode()



In [44]:
df_sorted.shape, df_processed.shape

((176, 128), (2401, 112))

In [45]:
index_number = 6268
df_processed.loc[df_processed.index == index_number]

,username,last_login_days,golive_days,last_month_total,mtd_total,last_month_open,mtd_open,last_month_closed,mtd_closed,last_month_age_hours,...,licence_registration_Day,licence_registration_Dayofweek,licence_registration_Dayofyear,licence_registration_Is_month_end,licence_registration_Is_month_start,licence_registration_Is_quarter_end,licence_registration_Is_quarter_start,licence_registration_Is_year_end,licence_registration_Is_year_start,licence_registration_Elapsed
6268,N065553,197.05,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,False,False,False,False,False,False,-9223372037


In [46]:
df_data.loc[df_data.index == index_number]

,username,last_login_days,golive_days,last_month_total,mtd_total,last_month_open,mtd_open,last_month_closed,mtd_closed,last_month_age_hours,...,churn_concern_cases_open_na,total_churn_concern_cases_age_avg_na,licence_registration_Year_na,licence_registration_Month_na,licence_registration_Week_na,licence_registration_Day_na,licence_registration_Dayofweek_na,licence_registration_Dayofyear_na,prediction,probability
6268,5837,197.05,42594.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,True,True,True,True,True,True,True,True,1,0.550882


In [47]:
df_sorted.loc[df_sorted.index == index_number]

,username,last_login_days,golive_days,last_month_total,mtd_total,last_month_open,mtd_open,last_month_closed,mtd_closed,last_month_age_hours,...,churn_concern_cases_open_na,total_churn_concern_cases_age_avg_na,licence_registration_Year_na,licence_registration_Month_na,licence_registration_Week_na,licence_registration_Day_na,licence_registration_Dayofweek_na,licence_registration_Dayofyear_na,prediction,probability
6268,5837,197.05,42594.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,True,True,True,True,True,True,True,True,1,0.550882


In [48]:
!wandb login dda97ba4df7f598b91affe3c806891df2e043ed7

wandb: Appending key for api.wandb.ai to your netrc file: /Users/justinhennessy/.netrc
Successfully logged in to Weights & Biases!


In [49]:
import time
seconds = time.time()

run_name = f"customer-inference-{int(seconds)}"
print(run_name)

wandb.init(project="neto-customer-churn", name=f"customer-inference-{int(seconds)}")


now = datetime.datetime.now()
array = []

for index, row in df_sorted.iterrows():
    output_string = f"{now}, " \
                    f"{df_processed.username.loc[df_processed.index == index].values[0]}, " \
                    f"{df.url.loc[df.index == index].values[0]}, " \
                    f"{df_raw.plan.loc[df_raw.index == index].values[0]}, , " \
                    f"{row.probability}, , " \
                    f"{df_raw.licence_status.loc[df_raw.index == index].values[0]}"
    print(output_string)
    array.append([output_string])
    
with open('./wandb/predictions.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(array)

writeFile.close()
wandb.save("predictions.csv")

customer-inference-1589279216


wandb: Wandb version 0.8.35 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


2020-05-12 20:26:57.325817, L006318, anaconda.neto.com.au, CUSTOM XL, , 0.7097614639503907, , LIVE
2020-05-12 20:26:57.325817, N016751, uk.store.xero.com, NETO XL, , 0.7084132352872672, , LIVE
2020-05-12 20:26:57.325817, N016753, us.store.xero.com, NETO XL, , 0.7079054825568752, , LIVE
2020-05-12 20:26:57.325817, N016755, au.store.xero.com, NETO XL, , 0.7077192127705617, , LIVE
2020-05-12 20:26:57.325817, N057171, www.grump.online, PLAN-360-TRANSITION, , 0.702160400434024, , LIVE
2020-05-12 20:26:57.325817, N065285, wildoutdoors.neto.com.au, PLAN-360-19Q1, , 0.6926644637353798, , LIVE
2020-05-12 20:26:57.325817, N050034, www.pleasurewarehouse.com.au, PLAN-S-19Q1, , 0.6812538089271604, , LIVE
2020-05-12 20:26:57.325817, N001896, www.outdoorfurniturecorporation.com.au, PLAN-S-19Q1, , 0.678833914981548, , LIVE
2020-05-12 20:26:57.325817, N051624, www.proflow.com.au, PLAN-S-19Q1, , 0.6786778667085622, , LIVE
2020-05-12 20:26:57.325817, N005686, www.additivefreeshop.com.au, PLAN-S-19Q1, , 0

[]

In [27]:
n_number = 'N007510'
utils.display_all(df_processed[df_processed.username == n_number])

,username,last_login_days,golive_days,last_month_total,mtd_total,last_month_open,mtd_open,last_month_closed,mtd_closed,last_month_age_hours,mtd_age_hours,month_total_avg,open_month_avg,closed_month_avg,age_hours_month_avg,interactions_total,interactions_completed,interactions_no_response,interactions_no_onboarding,interactions_completed_training,churn_concern_cases_total,churn_concern_cases_open,total_churn_concern_cases_age_avg,churned,plan_not_set,days_active_modified,customer_account_status_Good,customer_account_status_PoorStanding,plan_BASIC,plan_BRONZE,plan_BUSINESS,plan_BUSINESS +,plan_BUSINESS ONE,plan_CUSTOM XL,plan_ENTERPRISE,plan_GOLD,plan_GROW,plan_L&S - PREMIUM,plan_L&S - STANDARD,plan_L&S - ULTIMATE,plan_LIST N SEND SILVER,plan_LISTNSEND,plan_NETO XL,plan_NETO XXL,plan_ON-HOLD-19Q1,plan_PLAN-360-17Q2,plan_PLAN-360-19Q1,plan_PLAN-360-TRANSITION,plan_PLAN-360-US,plan_PLAN-L,plan_PLAN-L-17Q2,plan_PLAN-L-19Q1,plan_PLAN-L-US,plan_PLAN-M,plan_PLAN-M-17Q2,plan_PLAN-M-19Q1,plan_PLAN-M-US,plan_PLAN-S,plan_PLAN-S-17Q2,plan_PLAN-S-19Q1,plan_PLAN-S-US,plan_PLAN-XL-19Q1,plan_PLAN-XS,plan_PLAN-XS-17Q2,plan_PREMIER,plan_PREMIUM,plan_SILVER,plan_STANDARD,plan_START,plan_START - LEGACY,plan_TELSTRA - ONLINE STORE L,plan_TELSTRA - ONLINE STORE M,plan_TELSTRA - ONLINE STORE S,plan_TELSTRA - POS S,plan_TELSTRA - PREMIUM,plan_TELSTRA - STANDARD,plan_ULTIMATE,plan_ULTIMATE PLUS,plan_no_plan,nps_no_data,nps_detractor,nps_passive,nps_promoter,last_login_categories_day,last_login_categories_few_days,last_login_categories_week,last_login_categories_fortnight,last_login_categories_month,last_login_categories_month+,days_active_no_data,days_active_day,days_active_week,days_active_fortnight,days_active_month,days_active_3months,days_active_6months,days_active_1year,days_active_2years,days_active_2years+,licence_registration_Year,licence_registration_Month,licence_registration_Week,licence_registration_Day,licence_registration_Dayofweek,licence_registration_Dayofyear,licence_registration_Is_month_end,licence_registration_Is_month_start,licence_registration_Is_quarter_end,licence_registration_Is_quarter_start,licence_registration_Is_year_end,licence_registration_Is_year_start,licence_registration_Elapsed
973,N007510,197.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,-9223372037


In [28]:
from eli5 import show_prediction
loc_address = df_processed.index[df_processed.username == n_number].values[0]
df_data = df_data.drop(columns=['prediction', 'probability'])
show_prediction(churn_model, df_data.loc[loc_address], show_feature_values=True)